# Querying Metadata with the REST API

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `ENDPOINT_URL` to the location of the REST API.

In [1]:
import pandas as pd
import requests
import yaml

with open('config.yml') as file_handle:
    config = yaml.load(file_handle, yaml.FullLoader)

ENDPOINT_URL = config['rest_api']

Below we print the location of the endpoint we will query in this notebook:

In [2]:
print(f"REST API Endpoint: {ENDPOINT_URL}")

REST API Endpoint: http://mastapp.site/json


## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from get metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [3]:
response = requests.get(f'{ENDPOINT_URL}/shots')
result = response.json()
print(f"Query returned status code: {response.status_code}")

Query returned status code: 200


The shots endpoint returns a JSON payload with a list of shots. Let's look at the first element from the payload:

In [4]:
result['items'][0]

{'shot_id': 11695,
 'uuid': '219e5d59-b9dc-584f-b83b-018c47e8739d',
 'url': 's3://mast/level1/shots/11695.zarr',
 'timestamp': '2004-12-13T11:54:00',
 'preshot_description': '\n0.1T TF SHOT\n',
 'postshot_description': '\nOK\n',
 'campaign': 'M5',
 'reference_shot': None,
 'scenario': None,
 'heating': None,
 'pellets': False,
 'rmp_coil': None,
 'current_range': None,
 'divertor_config': 'Conventional',
 'plasma_shape': None,
 'comissioner': None,
 'facility': 'MAST',
 'cpf_p03249': None,
 'cpf_p04673': None,
 'cpf_p04674': None,
 'cpf_p04675': None,
 'cpf_p04676': None,
 'cpf_p04677': None,
 'cpf_p04678': None,
 'cpf_p04679': None,
 'cpf_p04680': None,
 'cpf_p04681': None,
 'cpf_p04833': None,
 'cpf_p04993': None,
 'cpf_p05007': None,
 'cpf_p05008': None,
 'cpf_p05009': None,
 'cpf_p05010': None,
 'cpf_p05011': None,
 'cpf_p05015': None,
 'cpf_p05016': None,
 'cpf_p05017': None,
 'cpf_p05025': None,
 'cpf_p05027': None,
 'cpf_p05028': None,
 'cpf_p05029': None,
 'cpf_p05030': None,
 

Each item in the list is a json object. This contains the meta-data items that corresponded to our query. In this case, each item contains information about a different MAST shot. Each item has lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://mast-app.site/redoc


Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe directly from the endpoint data

In [5]:
df = pd.read_json(f'{ENDPOINT_URL}/shots')['items']
df = pd.DataFrame(df.to_list())
df.head()

,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,11695,219e5d59-b9dc-584f-b83b-018c47e8739d,s3://mast/level1/shots/11695.zarr,2004-12-13T11:54:00,\n0.1T TF SHOT\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,11696,9ba69c96-34ac-5cd4-a1b8-8e9d7a44f871,s3://mast/level1/shots/11696.zarr,2004-12-13T12:07:00,\nSTANDARD 0.3T TF SHOT\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,11697,6ecd5c59-6d16-5a0f-a715-be31a88ca34e,s3://mast/level1/shots/11697.zarr,2004-12-13T12:19:00,\nRAISE TO 0.5T\n,"\nOK, ALARMS ARE LOWER\n",M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,11698,94366fda-b75e-5046-bb44-401844c563d8,s3://mast/level1/shots/11698.zarr,2004-12-13T12:31:00,\nRAISE TO .56T\n,\nSTILL ALARMS BUT LOWER AGAIN\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,11699,0e0f522d-a17f-5575-9bd0-d931231e8a71,s3://mast/level1/shots/11699.zarr,2004-12-13T12:45:00,\nRAISE TO .58T\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned. For example, we can return all shots for the `M9` campaign by using the approrpiate query string.

For example, we can query for everything from the `M9` campaign by adding `?campaign=M9` to our query string.

In [6]:
df = pd.read_json(f"{ENDPOINT_URL}/shots?campaign=M9")['items']
df = pd.DataFrame(df.to_list())
df.head()

,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,11695,219e5d59-b9dc-584f-b83b-018c47e8739d,s3://mast/level1/shots/11695.zarr,2004-12-13T11:54:00,\n0.1T TF SHOT\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,11696,9ba69c96-34ac-5cd4-a1b8-8e9d7a44f871,s3://mast/level1/shots/11696.zarr,2004-12-13T12:07:00,\nSTANDARD 0.3T TF SHOT\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,11697,6ecd5c59-6d16-5a0f-a715-be31a88ca34e,s3://mast/level1/shots/11697.zarr,2004-12-13T12:19:00,\nRAISE TO 0.5T\n,"\nOK, ALARMS ARE LOWER\n",M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,11698,94366fda-b75e-5046-bb44-401844c563d8,s3://mast/level1/shots/11698.zarr,2004-12-13T12:31:00,\nRAISE TO .56T\n,\nSTILL ALARMS BUT LOWER AGAIN\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,11699,0e0f522d-a17f-5575-9bd0-d931231e8a71,s3://mast/level1/shots/11699.zarr,2004-12-13T12:45:00,\nRAISE TO .58T\n,\nOK\n,M5,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response and corresponds to [RFC 8288](https://datatracker.ietf.org/doc/html/rfc8288). The response contains cursors at the bottom of the response:

 - `current_page` - this is the cursor relating to the current page of results.
 - `next_page` - this is the cursor for the next page of results.
 - `previous_page` - this is the cursor for the previous page of results, will result in `null` if on the first page.

You can control the pagination using the following options as query arguments:
 - `cursor=xx` to set the cursor for the displayed set of results
 - `size=xx` to set the number of results returned by each page.

In [7]:
response = requests.get(f'{ENDPOINT_URL}/shots?size=2&cursor=Pmk6MzAxMTE%3D')
result = response.json()
headers = response.headers

print("Current page cursor", result['current_page'])
print("Next page cursor", result['next_page'])
print("Previous page cursor", result['previous_page'])

df = pd.DataFrame(result)
df

Current page cursor Pmk6MzAxMTE%3D
Next page cursor Pmk6MzAxMTM%3D
Previous page cursor PGk6MzAxMTI%3D


,items,total,current_page,current_page_backwards,previous_page,next_page
0,"{'shot_id': 30112, 'uuid': 'eefe2467-69b0-5fe7...",None,Pmk6MzAxMTE%3D,PGk6MzAxMTQ%3D,PGk6MzAxMTI%3D,Pmk6MzAxMTM%3D
1,"{'shot_id': 30113, 'uuid': '742c78fa-3811-5404...",None,Pmk6MzAxMTE%3D,PGk6MzAxMTQ%3D,PGk6MzAxMTI%3D,Pmk6MzAxMTM%3D
